In [29]:
%%capture
!pip install transformers

In [2]:
from transformers import TFPegasusForConditionalGeneration, PegasusTokenizerFast
model_name = "thonyyy/pegasus-indonesian-base_finetune"
model = TFPegasusForConditionalGeneration.from_pretrained(model_name)
tokenizer = PegasusTokenizerFast.from_pretrained(model_name)

In [6]:
import re
import unicodedata
def text_cleaning(input_string):
    # Main function to clean text, removes link, bullet point, non ASCII char, news headline, parantheses,
    # punctuation except "," and ".", numbers with dot (enumerating), extra whitespaces, too short sentences.
    lowercase = input_string.lower()
    remove_link = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)', '', lowercase).replace("&amp;","&")
    remove_bullet = "\n".join([T for T in remove_link.split('\n') if '•' not in T and "baca juga:" not in T])
    remove_accented = unicodedata.normalize('NFKD', remove_bullet).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    remove_parentheses = re.sub("([\(\|]).*?([\)\|])", "\g<1>\g<2>", remove_accented)
    remove_punc = re.sub(r"[^\w\d.\s]+",' ', remove_parentheses)
    remove_num_dot = re.sub(r"(?<=\d)\.|\.(?=\d)|(?<=#)\.","", remove_punc)
    remove_extra_whitespace =  re.sub(r'^\s*|\s\s*', ' ', remove_num_dot).strip()
    return ".".join([s for s in remove_extra_whitespace.strip().split('.') if len(s.strip())>10]).replace("_","")

In [7]:
sample_article="""
Dana Moneter Internasional (IMF) menilai Indonesia telah menunjukkan pemulihan ekonomi yang baik pasca pandemi melalui kinerja makroekonomi yang kuat, didukung penerapan kebijakan moneter dan fiskal secara berhati-hati. Kebijakan forward looking dan sinergi telah berhasil membawa Indonesia menghadapi tantangan global pada tahun 2022 dengan pertumbuhan yang sehat, tekanan inflasi yang menurun, dan sistem keuangan yang stabil.  Bank Indonesia menyambut baik hasil asesmen IMF atas perekonomian Indonesia dalam laporan Article IV Consultation tahun 2023 yang ​dirilis hari ini (26/6).
Dewan Direktur IMF menyampaikan apresiasi dan catatan positif terhadap berbagai kebijakan yang ditempuh otoritas Indonesia selama tahun 2022. Pertama, keberhasilan otoritas untuk kembali kepada batas maksimal defisit fiskal 3%, lebih cepat dari yang diperkirakan dan komitmen otoritas untuk menerapkan disiplin fiskal. Kedua, penerapan kebijakan moneter yang memadai untuk menjaga stabilitas harga. Ketiga, ketahanan sektor keuangan yang tetap terjaga. Keempat, penerapan UU Cipta Kerja serta UU Pengembangan dan Penguatan Sektor Keuangan, dengan memastikan implementasi yang tepat dan keberlanjutan momentum reformasi untuk mendorong kemudahan berinvestasi, meningkatkan pendalaman pasar keuangan, dan memitigasi dampak scarring dari pandemi. Kelima, strategi diversifikasi Indonesia yang fokus pada upaya hilirisasi dalam rangka meningkatkan nilai tambah ekspor. Keenam, komitmen otoritas untuk mengurangi emisi  gas rumah kaca dan deforestasi.
"""

In [11]:
x = tokenizer(text_cleaning(t), return_tensors = 'tf')
y = model.generate(**x)
tokenizer.batch_decode(y, skip_special_tokens=True)